In [4]:
import arcpy
from arcpy.sa import *

# Set environment
arcpy.env.overwriteOutput = True

# ✅ Set your workspace to a File Geodatabase where outputs will be stored
arcpy.env.workspace = r"C:\Path\To\Your\Project.gdb"  # <-- Update this path!

# ✅ Input raster names (These must exist in the workspace)
dem_raster = "DigitalElevationModel"     # Ground DEM
building_dsm_raster = "BuildingDSM"      # DSM extracted from buildings only

# ✅ Load the rasters
dem = Raster(dem_raster)
building_dsm = Raster(building_dsm_raster)

# ✅ Combine DEM and BuildingDSM
# This will take the building heights and add them to the ground elevation
combined_surface = Con(IsNull(building_dsm), dem, building_dsm)

# ✅ Save the combined surface raster
combined_surface_name = "CombinedSurfaceDSM"
combined_surface.save(combined_surface_name)

print(f"✅ Combined surface raster '{combined_surface_name}' has been created and saved to your workspace.")


RuntimeError: ERROR 000875: Output raster: C:\Path\To\Your\Project.gdb\CombinedSurfaceDSM's workspace is an invalid output workspace.

In [6]:
import arcpy
from arcpy.sa import *

# ✅ Auto-detect the current ArcGIS Pro workspace (linked to your current map/project)
workspace = arcpy.env.workspace

# Check if the workspace is properly set in ArcGIS Pro
if not workspace:
    raise Exception("⚠️ Workspace is not set. Please specify an active workspace in ArcGIS Pro before running this script.")

# Show the current workspace for confirmation
print(f"📂 Active workspace: {workspace}")

# ✅ Allow overwriting outputs
arcpy.env.overwriteOutput = True

# ✅ Input raster names
dem_raster = "DigitalElevationModel"     # Ground DEM name (already loaded in your workspace)
building_dsm_raster = "BuildingDSM"      # DSM of buildings (extracted building heights)

# ✅ Load the rasters
dem = Raster(dem_raster)
building_dsm = Raster(building_dsm_raster)

# ✅ Combine DEM and BuildingDSM
# If buildingDSM is null (no building), use the DEM; otherwise, use the buildingDSM (building heights already included)
combined_surface = Con(IsNull(building_dsm), dem, building_dsm)

# ✅ Save the combined raster in the workspace geodatabase
combined_surface_name = "CombinedSurfaceDSM"
combined_surface.save(combined_surface_name)

# ✅ Success message
print(f"✅ Combined surface raster '{combined_surface_name}' has been created and saved to: {workspace}")


📂 Active workspace: C:\Path\To\Your\Project.gdb


RuntimeError: ERROR 000875: Output raster: C:\Path\To\Your\Project.gdb\CombinedSurfaceDSM's workspace is an invalid output workspace.

In [7]:
import arcpy
from arcpy.sa import *

# ✅ Automatically detect the current ArcGIS Pro workspace (already set)
workspace = arcpy.env.workspace

# Verify workspace is set
if not workspace:
    raise Exception("⚠️ Workspace is not set. Please specify an active workspace in ArcGIS Pro.")

# Show the workspace for verification
print(f"📂 Current workspace: {workspace}")

arcpy.env.overwriteOutput = True

# ✅ Input elevation surface (combined DEM + buildings)
elevation_surface = "CombinedSurfaceDSM"  # This is the combined DSM you created earlier

# ✅ Observer and target heights above ground/building in meters (3.5 feet = 1.07 meters)
observer_height = 1.07
target_height = 1.07

# ✅ Spatial reference (WGS 84 for input coords, NAD83 UTM 18N for projections)
spatial_ref_wgs84 = arcpy.SpatialReference(4326)   # Latitude/Longitude
spatial_ref_projected = arcpy.SpatialReference(26918)  # NAD 1983 UTM Zone 18N

# ✅ Observer-Target Pairs (First Two Examples)
observer_target_pairs = [
    ("1a", (38.88001799190699, -77.14628518506876), (38.879350486706414, -77.1457651587032)),
    ("1b", (38.88001799190699, -77.14626760480989), (38.879350486706414, -77.1457651587032))
]

# ✅ Helper function to convert meters to feet
def meters_to_feet(meters):
    return meters * 3.28084

# ✅ Store visibility results
visibility_results = []

# ✅ Loop through each pair
for name, (obs_lat, obs_lon), (tar_lat, tar_lon) in observer_target_pairs:
    print(f"\n▶️ Processing Pair: {name}")

    # Unique feature class names
    line_fc = f"LOS_Line_{name}"
    line_of_sight_fc = f"LOS_Results_{name}"
    obstruction_fc = f"Obstruction_{name}"

    # ✅ Create a new polyline feature class for the LOS line
    arcpy.management.CreateFeatureclass(workspace, line_fc, "POLYLINE", spatial_reference=spatial_ref_wgs84)

    # ✅ Insert observer-to-target line (with height offsets)
    with arcpy.da.InsertCursor(line_fc, ["SHAPE@"]) as cursor:
        polyline = arcpy.Polyline(
            arcpy.Array([
                arcpy.Point(obs_lon, obs_lat, observer_height),  # Observer point with height
                arcpy.Point(tar_lon, tar_lat, target_height)     # Target point with height
            ]),
            spatial_ref_wgs84
        )
        cursor.insertRow([polyline])

    # ✅ Run the Line of Sight tool using the combined surface
    try:
        arcpy.ddd.LineOfSight(
            in_surface=elevation_surface,
            in_line_feature_class=line_fc,
            out_los_feature_class=line_of_sight_fc,
            out_obstruction_feature_class=obstruction_fc,
            use_curvature="NO_CURVATURE",
            use_refraction="NO_REFRACTION",
            refraction_factor=0.13,
            pyramid_level_resolution=0
        )
        print(f"✅ Line of Sight run complete for {name}")
    except Exception as e:
        print(f"❌ Error processing {name}: {e}")
        continue

    # ✅ Initialize obstruction distance variable (in feet)
    first_obstruction_distance_ft = None

    # ✅ Extract LOS results (TarIsVis and Shape_Length fields)
    if arcpy.Exists(line_of_sight_fc):
        with arcpy.da.SearchCursor(line_of_sight_fc, ["TarIsVis", "Shape_Length"]) as cursor:
            for row in cursor:
                tar_is_visible = row[0]  # 0 = Obstructed, 1 = Visible
                shape_length_ft = meters_to_feet(row[1])  # Convert distance to feet
                visibility_results.append((name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft))

    # ✅ Extract distance to the first obstruction (if obstruction exists)
    if arcpy.Exists(obstruction_fc):
        desc = arcpy.Describe(obstruction_fc)
        obstruction_spatial_ref = desc.spatialReference  # Spatial reference of obstruction layer

        with arcpy.da.SearchCursor(obstruction_fc, ["SHAPE@"]) as cursor:
            for obstruction in cursor:
                obstruction_geom = obstruction[0]
                observer_geom = arcpy.PointGeometry(arcpy.Point(obs_lon, obs_lat), spatial_ref_wgs84)

                # Project to obstruction spatial reference to calculate distance
                obstruction_projected = obstruction_geom.projectAs(obstruction_spatial_ref)
                observer_projected = observer_geom.projectAs(obstruction_spatial_ref)

                # Compute distance in feet
                first_obstruction_distance_ft = meters_to_feet(observer_projected.distanceTo(obstruction_projected))
                break  # Only the first obstruction point is considered

        # Update the result with obstruction distance
        if visibility_results:
            last_result = visibility_results[-1]
            visibility_results[-1] = (last_result[0], last_result[1], last_result[2], first_obstruction_distance_ft)

# ✅ Print LOS analysis results
if visibility_results:
    print("\n🔹 Line of Sight Analysis Results for First 2 Pairs (in feet):")
    for name, tar_is_visible, shape_length_ft, obstruction_distance_ft in visibility_results:
        status = "Visible" if tar_is_visible == 1 else "Obstructed"
        obs_distance_text = f", First Obstruction at {obstruction_distance_ft:.2f} feet" if obstruction_distance_ft else ""
        print(f"{name}: {status}, Total Length = {shape_length_ft:.2f} feet{obs_distance_text}")
else:
    print("⚠️ No sight lines were generated.")

print("\n✅ Line of Sight analyses completed for the first two observer-target pairs.")


📂 Current workspace: C:\Path\To\Your\Project.gdb

▶️ Processing Pair: 1a


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Feature Class Location: Dataset C:\Path\To\Your\Project.gdb does not exist or is not supported
Failed to execute (CreateFeatureclass).


In [8]:
import arcpy

# Automatically detect the current ArcGIS Pro workspace
workspace = arcpy.env.workspace
if not workspace:
    raise Exception("Workspace is not set. Please specify an active workspace in ArcGIS Pro.")

arcpy.env.overwriteOutput = True  # Allow overwriting existing outputs

# Define input elevation surface (Digital Surface Model)
elevation_surface = "combined_surface"  # Ensure this DSM exists in your workspace

# Define observer and target height above the surface (3.5 feet = 1.07 meters)
observer_height = 1.07  # in meters
target_height = 1.07  # in meters

# Set spatial reference to WGS 84 (lat/lon)
spatial_ref_wgs84 = arcpy.SpatialReference(4326)  # EPSG:4326

# Set spatial reference to NAD 1983 (2011) UTM Zone 18N
spatial_ref_projected = arcpy.SpatialReference(26918)

# Convert meters to feet function
def meters_to_feet(meters):
    return meters * 3.28084

# Observer and target coordinate pairs (name, observer, target)
observer_target_pairs = [
    # Format: (name, (observer_lat, observer_lon), (target_lat, target_lon))
    ("1_SDR", (38.86922736835016, -77.23329317992511), (38.87008408808786, -77.23302603408561)),
    ("1_SDL", (38.86922736835016, -77.23329317992511), (38.86854902979591, -77.23380643217587)),
]

# Store LOS results
visibility_results = []

# Process each observer-target pair
for name, (obs_lat, obs_lon), (tar_lat, tar_lon) in observer_target_pairs:

    # Define unique names for each pair
    line_fc = f"LOS_Line_{name}"
    line_of_sight_fc = f"LOS_Results_{name}"
    obstruction_fc = f"Obstruction_{name}"

    # Create a new polyline feature class for the LOS line
    arcpy.management.CreateFeatureclass(workspace, line_fc, "POLYLINE", spatial_reference=spatial_ref_wgs84)

    # Insert observer-to-target line with elevation applied
    with arcpy.da.InsertCursor(line_fc, ["SHAPE@"]) as cursor:
        polyline = arcpy.Polyline(
            arcpy.Array([
                arcpy.Point(obs_lon, obs_lat, observer_height),  # Observer point
                arcpy.Point(tar_lon, tar_lat, target_height)  # Target point
            ]),
            spatial_ref_wgs84
        )
        cursor.insertRow([polyline])

    # Run Line of Sight Tool using the DSM as input surface
    arcpy.ddd.LineOfSight(
        in_surface=elevation_surface,
        in_line_feature_class=line_fc,
        out_los_feature_class=line_of_sight_fc,
        out_obstruction_feature_class=obstruction_fc,
        use_curvature="NO_CURVATURE",
        use_refraction="NO_REFRACTION",
        refraction_factor=0.13,
        pyramid_level_resolution=0
    )

    # Initialize obstruction distance variable
    first_obstruction_distance_ft = None  

    # Extract LOS results (TarIsVis and Shape_Length fields)
    if arcpy.Exists(line_of_sight_fc):
        with arcpy.da.SearchCursor(line_of_sight_fc, ["TarIsVis", "Shape_Length"]) as cursor:
            for row in cursor:
                tar_is_visible = row[0]  # 0 = Obstructed, 1 = Visible
                shape_length_ft = meters_to_feet(row[1])
                visibility_results.append((name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft))

    # Extract first obstruction point distance (if obstructed)
    if arcpy.Exists(obstruction_fc):
        desc = arcpy.Describe(obstruction_fc)
        obstruction_spatial_ref = desc.spatialReference

        with arcpy.da.SearchCursor(obstruction_fc, ["SHAPE@"]) as cursor:
            for obstruction in cursor:
                obstruction_geom = obstruction[0]
                observer_geom = arcpy.PointGeometry(arcpy.Point(obs_lon, obs_lat), spatial_ref_wgs84)

                # Project both points to the obstruction spatial reference
                obstruction_projected = obstruction_geom.projectAs(obstruction_spatial_ref)
                observer_projected = observer_geom.projectAs(obstruction_spatial_ref)

                # Compute distance in feet
                first_obstruction_distance_ft = meters_to_feet(observer_projected.distanceTo(obstruction_projected))
                break

        # Update result with obstruction distance
        visibility_results[-1] = (name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft)

# Print LOS analysis results
if visibility_results:
    print("\n🔹 Line of Sight Analysis Results (in feet):")
    for name, tar_is_visible, shape_length_ft, obstruction_distance_ft in visibility_results:
        status = "Visible" if tar_is_visible == 1 else "Obstructed"
        obs_text = f", First Obstruction at {obstruction_distance_ft:.2f} ft" if obstruction_distance_ft else ""
        print(f"{name}: {status}, LOS Length = {shape_length_ft:.2f} ft{obs_text}")
else:
    print("⚠ No sight lines were generated.")

print("\n✅ LOS Analysis Complete. Results stored in your ArcGIS workspace.")


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Feature Class Location: Dataset C:\Path\To\Your\Project.gdb does not exist or is not supported
Failed to execute (CreateFeatureclass).


In [10]:
import arcpy

# ✅ SET YOUR WORKSPACE PATH HERE (YourProject.gdb)
workspace = r"C:\Users\shara\OneDrive\Documents\ArcGIS\Projects\Latestt\Latestt.gdb"  # Update this path
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# ✅ Validate workspace exists
if not arcpy.Exists(workspace):
    raise Exception(f"⚠️ Workspace not found: {workspace}")

print(f"📂 Current workspace set to: {workspace}")

# ✅ Define the input surface (combined DSM + DTM raster created earlier)
elevation_surface = "combined_surface"  # It should already exist in your geodatabase or map

# Observer and target height above the surface (3.5 feet = 1.07 meters)
observer_height = 1.07  # meters
target_height = 1.07  # meters

# Set spatial references
spatial_ref_wgs84 = arcpy.SpatialReference(4326)   # WGS 84 (Geographic Coordinate System)
spatial_ref_projected = arcpy.SpatialReference(26918)  # NAD 1983 UTM Zone 18N

# ✅ Convert meters to feet
def meters_to_feet(meters):
    return meters * 3.28084

# ✅ Define observer-target coordinate pairs (as per your example)
observer_target_pairs = [
    ("1_SDR", (38.86922736835016, -77.23329317992511), (38.87008408808786, -77.23302603408561)),
    ("1_SDL", (38.86922736835016, -77.23329317992511), (38.86854902979591, -77.23380643217587)),

    ("2_SDR", (38.877763340549585, -77.14349160147528), (38.87768921498809, -77.14441575933215)),
    ("2_SDL", (38.87775079043092, -77.14349729129944), (38.87768921498809, -77.14441575933215)),

    ("3_SDR", (38.88002159730838, -77.14628291494252), (38.87936086777408, -77.14576590252301)),
    ("3_SDL", (38.88001846531143, -77.14626816279392), (38.87936086777408, -77.14576590252301)),

    ("4_SDR", (38.8795982084526, -77.14576896267089), (38.88027072740311, -77.14603777394164)),
    ("4_SDL", (38.87959386306809, -77.14578411375457), (38.88027072740311, -77.14603777394164)),

    ("5_SDR", (38.87832209089586, -77.14431720264261), (38.877690269745386, -77.14429227968809)),
    ("5_SDL", (38.878315067567634, -77.14429915896426), (38.877690269745386, -77.14429227968809)),

    ("6_SDR", (38.87696443624974, -77.14201769147124), (38.876266531397654, -77.1420828326032)),
    ("6_SDL", (38.87695822832742, -77.14200014811117), (38.876266531397654, -77.1420828326032)),

    ("7_SDR", (38.87670860516591, -77.14111794320542), (38.876002765804714, -77.1412471563881)),
    ("7_SDL", (38.87670338492754, -77.14110319105681), (38.876002765804714, -77.1412471563881)),

    ("8_SDR", (38.88001799190699, -77.14628518506876), (38.879350486706414, -77.1457651587032)),
    ("8_SDL", (38.88001799190699, -77.14626760480989), (38.879350486706414, -77.1457651587032)),

    ("9_SDR", (38.877804801692434, -77.14435959987517), (38.87765728939592, -77.14524376446222)),
    ("9_SDL", (38.87779300677906, -77.1443611947261), (38.87765728939592, -77.14524376446222)),

    ("10_SDR", (38.878390040587284, -77.14135353872355), (38.877646142443616, -77.14144578288274)),
    ("10_SDL", (38.8783856951289, -77.14134237476715), (38.877646142443616, -77.14144578288274))
]

# ✅ Store LOS results for reporting
visibility_results = []

# ✅ Iterate through each observer-target pair
for name, (obs_lat, obs_lon), (tar_lat, tar_lon) in observer_target_pairs:
    print(f"\n▶️ Processing Pair: {name}")

    # Feature class names for LOS analysis
    line_fc = f"LOS_Line_{name}"
    los_results_fc = f"LOS_Results_{name}"
    obstruction_fc = f"LOS_Obstruction_{name}"

    # Create polyline feature class for observer-to-target line
    arcpy.management.CreateFeatureclass(
        workspace,
        line_fc,
        "POLYLINE",
        spatial_reference=spatial_ref_wgs84
    )

    # Insert line geometry with observer and target points
    with arcpy.da.InsertCursor(line_fc, ["SHAPE@"]) as cursor:
        polyline = arcpy.Polyline(
            arcpy.Array([
                arcpy.Point(obs_lon, obs_lat, observer_height),  # Observer point with height offset
                arcpy.Point(tar_lon, tar_lat, target_height)     # Target point with height offset
            ]),
            spatial_ref_wgs84
        )
        cursor.insertRow([polyline])

    # ✅ Run Line of Sight tool
    try:
        arcpy.ddd.LineOfSight(
            in_surface=elevation_surface,
            in_line_feature_class=line_fc,
            out_los_feature_class=los_results_fc,
            out_obstruction_feature_class=obstruction_fc,
            use_curvature="NO_CURVATURE",
            use_refraction="NO_REFRACTION",
            refraction_factor=0.13,
            pyramid_level_resolution=0
        )
        print(f"✅ LOS analysis complete for {name}")
    except Exception as e:
        print(f"❌ LOS analysis failed for {name}: {e}")
        continue  # Move to next pair if it fails

    # ✅ Initialize obstruction distance
    first_obstruction_distance_ft = None  

    # ✅ Extract LOS results (TarIsVis and Shape_Length fields)
    if arcpy.Exists(los_results_fc):
        with arcpy.da.SearchCursor(los_results_fc, ["TarIsVis", "Shape_Length"]) as cursor:
            for row in cursor:
                tar_is_visible = row[0]  # 0 = Obstructed, 1 = Visible
                shape_length_ft = meters_to_feet(row[1])  # Convert to feet
                visibility_results.append((name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft))

    # ✅ Extract obstruction points if they exist
    if arcpy.Exists(obstruction_fc):
        desc = arcpy.Describe(obstruction_fc)
        obstruction_spatial_ref = desc.spatialReference  # usually NAD83 or the same CRS as input

        with arcpy.da.SearchCursor(obstruction_fc, ["SHAPE@"]) as cursor:
            for obstruction in cursor:
                obstruction_geom = obstruction[0]  # Obstruction point geometry
                observer_geom = arcpy.PointGeometry(arcpy.Point(obs_lon, obs_lat), spatial_ref_wgs84)

                # Project both points to match CRS for distance calculation
                obstruction_projected = obstruction_geom.projectAs(obstruction_spatial_ref)
                observer_projected = observer_geom.projectAs(obstruction_spatial_ref)

                # Compute distance in feet from observer to first obstruction
                first_obstruction_distance_ft = meters_to_feet(observer_projected.distanceTo(obstruction_projected))
                break  # Only process the first obstruction point

        # ✅ Update the latest result entry with the first obstruction distance
        if visibility_results:
            last_result = visibility_results[-1]
            visibility_results[-1] = (last_result[0], last_result[1], last_result[2], first_obstruction_distance_ft)

# ✅ Print final LOS results
if visibility_results:
    print("\n🔹 Line of Sight Analysis Results (in feet):")
    for name, tar_is_visible, shape_length_ft, obstruction_distance_ft in visibility_results:
        status = "Visible" if tar_is_visible == 1 else "Obstructed"
        obs_text = f", First Obstruction at {obstruction_distance_ft:.2f} ft" if obstruction_distance_ft else ""
        print(f"{name}: {status}, LOS Length = {shape_length_ft:.2f} ft{obs_text}")
else:
    print("⚠ No LOS results available!")

print("\n✅ Line of Sight analyses completed for all observer-target pairs.")


📂 Current workspace set to: C:\Users\shara\OneDrive\Documents\ArcGIS\Projects\Latestt\Latestt.gdb

▶️ Processing Pair: 1_SDR
✅ LOS analysis complete for 1_SDR

▶️ Processing Pair: 1_SDL
✅ LOS analysis complete for 1_SDL

▶️ Processing Pair: 2_SDR
✅ LOS analysis complete for 2_SDR

▶️ Processing Pair: 2_SDL
✅ LOS analysis complete for 2_SDL

▶️ Processing Pair: 3_SDR
✅ LOS analysis complete for 3_SDR

▶️ Processing Pair: 3_SDL
✅ LOS analysis complete for 3_SDL

▶️ Processing Pair: 4_SDR
✅ LOS analysis complete for 4_SDR

▶️ Processing Pair: 4_SDL
✅ LOS analysis complete for 4_SDL

▶️ Processing Pair: 5_SDR
✅ LOS analysis complete for 5_SDR

▶️ Processing Pair: 5_SDL
✅ LOS analysis complete for 5_SDL

▶️ Processing Pair: 6_SDR
✅ LOS analysis complete for 6_SDR

▶️ Processing Pair: 6_SDL
✅ LOS analysis complete for 6_SDL

▶️ Processing Pair: 7_SDR
✅ LOS analysis complete for 7_SDR

▶️ Processing Pair: 7_SDL
✅ LOS analysis complete for 7_SDL

▶️ Processing Pair: 8_SDR
✅ LOS analysis comple

In [11]:
import arcpy

# ✅ SET YOUR WORKSPACE PATH HERE (YourProject.gdb)
workspace = r"C:\Users\shara\OneDrive\Documents\ArcGIS\Projects\Latestt\Latestt.gdb"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# ✅ Validate workspace exists
if not arcpy.Exists(workspace):
    raise Exception(f"⚠️ Workspace not found: {workspace}")

print(f"📂 Current workspace set to: {workspace}")

# ✅ Define the input surface (combined DSM + DTM raster created earlier)
elevation_surface = "combined_surface"  # Must exist in your GDB or map

# Observer and target height above the surface (3.5 feet = 1.07 meters)
observer_height = 1.07  # meters
target_height = 1.07  # meters

# Set spatial references
spatial_ref_wgs84 = arcpy.SpatialReference(4326)   # WGS 84
spatial_ref_projected = arcpy.SpatialReference(26918)  # NAD 1983 UTM Zone 18N

# ✅ Convert meters to feet
def meters_to_feet(meters):
    return meters * 3.28084

# ✅ Define observer-target coordinate pairs
observer_target_pairs = [
    ("1_SDR", (38.86922736835016, -77.23329317992511), (38.87008408808786, -77.23302603408561)),
    ("1_SDL", (38.86922736835016, -77.23329317992511), (38.86854902979591, -77.23380643217587)),

    ("2_SDR", (38.877763340549585, -77.14349160147528), (38.87768921498809, -77.14441575933215)),
    ("2_SDL", (38.87775079043092, -77.14349729129944), (38.87768921498809, -77.14441575933215)),

    ("3_SDR", (38.88002159730838, -77.14628291494252), (38.87936086777408, -77.14576590252301)),
    ("3_SDL", (38.88001846531143, -77.14626816279392), (38.87936086777408, -77.14576590252301)),

    ("4_SDR", (38.8795982084526, -77.14576896267089), (38.88027072740311, -77.14603777394164)),
    ("4_SDL", (38.87959386306809, -77.14578411375457), (38.88027072740311, -77.14603777394164)),

    ("5_SDR", (38.87832209089586, -77.14431720264261), (38.877690269745386, -77.14429227968809)),
    ("5_SDL", (38.878315067567634, -77.14429915896426), (38.877690269745386, -77.14429227968809)),

    ("6_SDR", (38.87696443624974, -77.14201769147124), (38.876266531397654, -77.1420828326032)),
    ("6_SDL", (38.87695822832742, -77.14200014811117), (38.876266531397654, -77.1420828326032)),

    ("7_SDR", (38.87670860516591, -77.14111794320542), (38.876002765804714, -77.1412471563881)),
    ("7_SDL", (38.87670338492754, -77.14110319105681), (38.876002765804714, -77.1412471563881)),

    ("8_SDR", (38.88001799190699, -77.14628518506876), (38.879350486706414, -77.1457651587032)),
    ("8_SDL", (38.88001799190699, -77.14626760480989), (38.879350486706414, -77.1457651587032)),

    ("9_SDR", (38.877804801692434, -77.14435959987517), (38.87765728939592, -77.14524376446222)),
    ("9_SDL", (38.87779300677906, -77.1443611947261), (38.87765728939592, -77.14524376446222)),

    ("10_SDR", (38.878390040587284, -77.14135353872355), (38.877646142443616, -77.14144578288274)),
    ("10_SDL", (38.8783856951289, -77.14134237476715), (38.877646142443616, -77.14144578288274))
]

# ✅ Store LOS results
visibility_results = []

# ✅ Iterate through each observer-target pair
for name, (obs_lat, obs_lon), (tar_lat, tar_lon) in observer_target_pairs:
    print(f"\n▶️ Processing Pair: {name}")

    # Feature class names for LOS analysis
    line_fc = f"LOS_Line_{name}"
    los_results_fc = f"LOS_Results_{name}"
    obstruction_fc = f"LOS_Obstruction_{name}"

    # Create polyline feature class for observer-to-target line
    arcpy.management.CreateFeatureclass(
        workspace,
        line_fc,
        "POLYLINE",
        spatial_reference=spatial_ref_wgs84
    )

    # Insert line geometry
    with arcpy.da.InsertCursor(line_fc, ["SHAPE@"]) as cursor:
        polyline = arcpy.Polyline(
            arcpy.Array([
                arcpy.Point(obs_lon, obs_lat, observer_height),
                arcpy.Point(tar_lon, tar_lat, target_height)
            ]),
            spatial_ref_wgs84
        )
        cursor.insertRow([polyline])

    # ✅ Run Line of Sight tool
    try:
        arcpy.ddd.LineOfSight(
            in_surface=elevation_surface,
            in_line_feature_class=line_fc,
            out_los_feature_class=los_results_fc,
            out_obstruction_feature_class=obstruction_fc,
            use_curvature="NO_CURVATURE",
            use_refraction="NO_REFRACTION",
            refraction_factor=0.13,
            pyramid_level_resolution=0
        )
        print(f"✅ LOS analysis complete for {name}")
    except Exception as e:
        print(f"❌ LOS analysis failed for {name}: {e}")
        continue

    # ✅ Initialize obstruction distance
    first_obstruction_distance_ft = None  

    # ✅ Extract LOS results
    if arcpy.Exists(los_results_fc):
        with arcpy.da.SearchCursor(los_results_fc, ["TarIsVis", "Shape_Length"]) as cursor:
            for row in cursor:
                tar_is_visible = row[0]
                shape_length_ft = meters_to_feet(row[1])
                visibility_results.append((name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft))

    # ✅ Extract obstruction distance
    if arcpy.Exists(obstruction_fc):
        desc = arcpy.Describe(obstruction_fc)
        obstruction_spatial_ref = desc.spatialReference

        with arcpy.da.SearchCursor(obstruction_fc, ["SHAPE@"]) as cursor:
            for obstruction in cursor:
                obstruction_geom = obstruction[0]
                observer_geom = arcpy.PointGeometry(arcpy.Point(obs_lon, obs_lat), spatial_ref_wgs84)

                obstruction_projected = obstruction_geom.projectAs(obstruction_spatial_ref)
                observer_projected = observer_geom.projectAs(obstruction_spatial_ref)

                first_obstruction_distance_ft = meters_to_feet(observer_projected.distanceTo(obstruction_projected))
                break  # Only take the first obstruction point

        # Update the last visibility_results entry
        if visibility_results:
            last_result = visibility_results[-1]
            visibility_results[-1] = (last_result[0], last_result[1], last_result[2], first_obstruction_distance_ft)

# ✅ Final LOS Analysis Results
if visibility_results:
    print("\n🔹 Final LOS Results (Feet):")
    for name, tar_is_visible, shape_length_ft, obstruction_distance_ft in visibility_results:
        status = "Visible" if tar_is_visible == 1 else "Obstructed"
        obstruction_text = f", First Obstruction at {obstruction_distance_ft:.2f} ft" if obstruction_distance_ft else ""
        print(f"{name}: {status}, LOS Length = {shape_length_ft:.2f} ft{obstruction_text}")
else:
    print("⚠ No LOS results available!")

print("\n✅ Line of Sight Analysis Completed for All Observer-Target Pairs!")


📂 Current workspace set to: C:\Users\shara\OneDrive\Documents\ArcGIS\Projects\Latestt\Latestt.gdb

▶️ Processing Pair: 1_SDR
✅ LOS analysis complete for 1_SDR

▶️ Processing Pair: 1_SDL
✅ LOS analysis complete for 1_SDL

▶️ Processing Pair: 2_SDR
✅ LOS analysis complete for 2_SDR

▶️ Processing Pair: 2_SDL
✅ LOS analysis complete for 2_SDL

▶️ Processing Pair: 3_SDR
✅ LOS analysis complete for 3_SDR

▶️ Processing Pair: 3_SDL
✅ LOS analysis complete for 3_SDL

▶️ Processing Pair: 4_SDR
✅ LOS analysis complete for 4_SDR

▶️ Processing Pair: 4_SDL
✅ LOS analysis complete for 4_SDL

▶️ Processing Pair: 5_SDR
✅ LOS analysis complete for 5_SDR

▶️ Processing Pair: 5_SDL
✅ LOS analysis complete for 5_SDL

▶️ Processing Pair: 6_SDR
✅ LOS analysis complete for 6_SDR

▶️ Processing Pair: 6_SDL
✅ LOS analysis complete for 6_SDL

▶️ Processing Pair: 7_SDR
✅ LOS analysis complete for 7_SDR

▶️ Processing Pair: 7_SDL
✅ LOS analysis complete for 7_SDL

▶️ Processing Pair: 8_SDR
✅ LOS analysis comple

In [12]:
import arcpy

# Automatically detect the current ArcGIS Pro workspace
workspace = arcpy.env.workspace
if not workspace:
    raise Exception("Workspace is not set. Please specify an active workspace in ArcGIS Pro.")

arcpy.env.overwriteOutput = True  # Allow overwriting existing outputs

# Define input elevation surface (Digital Surface Model)
elevation_surface = "combined_surface"  # Ensure this DSM exists in your workspace

# Define observer and target height above the surface (3.5 feet = 1.07 meters)
observer_height = 1.07  # in meters
target_height = 1.07  # in meters

# Set spatial reference to WGS 84 (lat/lon)
spatial_ref_wgs84 = arcpy.SpatialReference(4326)  # EPSG:4326

# Set spatial reference to NAD 1983 (2011) UTM Zone 18N
spatial_ref_projected = arcpy.SpatialReference(26918)

# Convert meters to feet function
def meters_to_feet(meters):
    return meters * 3.28084

# Observer and target coordinate pairs (name, observer, target)
observer_target_pairs = [
    # Format: (name, (observer_lat, observer_lon), (target_lat, target_lon))
    ("1_SDR", (38.86922736835016, -77.23329317992511), (38.87008408808786, -77.23302603408561)),
    ("1_SDL", (38.86922736835016, -77.23329317992511), (38.86854902979591, -77.23380643217587)),

    ("2_SDR", (38.869131257699586, -77.23279250838289), (38.869330738339485, -77.23376469489178)),
    ("2_SDL", (38.869131257699586, -77.23279250838289), (38.868781801428625, -77.23189061398745)),

    ("3_SDR", (38.868401296315014, -77.23392957904841), (38.86801675129652, -77.23458874982923)),
    ("3_SDL", (38.868401296315014, -77.23392957904841), (38.86774368899713, -77.23335362494895)),

    ("4_SDR", (38.86895020107385, -77.23223381995975), (38.869153521944455, -77.2332001786722)),
    ("4_SDL", (38.86895020107385, -77.23223381995975), (38.868734685920415, -77.2313267678459)),

    ("5_SDR", (38.870047, -77.232825), (38.87092017439365, -77.23263410670167)),
    ("5_SDL", (38.870047, -77.232825), (38.869339103045625, -77.23331403667311)),

    ("6_SDR", (38.870818351528165, -77.23245095581497), (38.87169263610585, -77.23226052607563)),
    ("6_SDL", (38.870818351528165, -77.23245095581497), (38.87008215877504, -77.23292297859463)),

    ("7_SDR", (38.873428, -77.231198), (38.87428956541312, -77.23149722227092)),
    ("7_SDL", (38.873428, -77.231198), (38.872631, -77.231681)),

    ("8_SDR", (38.87237692334737, -77.23043357600156), (38.8723089540676, -77.22929010100793)),
    ("8_SDL", (38.87237692334737, -77.23043357600156), (38.87269259675397, -77.23138846110841)),

    ("9_SDR", (38.872757181946106, -77.23148127391173), (38.87350208113631, -77.23147942407168)),
    ("9_SDL", (38.872757181946106, -77.23148127391173), (38.871975396357186, -77.23201771730614)),

    ("10_SDR", (38.8689334003095, -77.2240225533071), (38.86937491141229, -77.22322741772842)),
    ("10_SDL", (38.8689334003095, -77.2240225533071), (38.86869549824453, -77.22499606357371)),
]

# Store LOS results
visibility_results = []

# Process each observer-target pair
for name, (obs_lat, obs_lon), (tar_lat, tar_lon) in observer_target_pairs:

    # Define unique names for each pair
    line_fc = f"LOS_Line_{name}"
    line_of_sight_fc = f"LOS_Results_{name}"
    obstruction_fc = f"Obstruction_{name}"

    # Create a new polyline feature class for the LOS line
    arcpy.management.CreateFeatureclass(workspace, line_fc, "POLYLINE", spatial_reference=spatial_ref_wgs84)

    # Insert observer-to-target line with elevation applied
    with arcpy.da.InsertCursor(line_fc, ["SHAPE@"]) as cursor:
        polyline = arcpy.Polyline(
            arcpy.Array([
                arcpy.Point(obs_lon, obs_lat, observer_height),  # Observer point
                arcpy.Point(tar_lon, tar_lat, target_height)  # Target point
            ]),
            spatial_ref_wgs84
        )
        cursor.insertRow([polyline])

    # Run Line of Sight Tool using the DSM as input surface
    arcpy.ddd.LineOfSight(
        in_surface=elevation_surface,
        in_line_feature_class=line_fc,
        out_los_feature_class=line_of_sight_fc,
        out_obstruction_feature_class=obstruction_fc,
        use_curvature="NO_CURVATURE",
        use_refraction="NO_REFRACTION",
        refraction_factor=0.13,
        pyramid_level_resolution=0
    )

    # Initialize obstruction distance variable
    first_obstruction_distance_ft = None  

    # Extract LOS results (TarIsVis and Shape_Length fields)
    if arcpy.Exists(line_of_sight_fc):
        with arcpy.da.SearchCursor(line_of_sight_fc, ["TarIsVis", "Shape_Length"]) as cursor:
            for row in cursor:
                tar_is_visible = row[0]  # 0 = Obstructed, 1 = Visible
                shape_length_ft = meters_to_feet(row[1])
                visibility_results.append((name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft))

    # Extract first obstruction point distance (if obstructed)
    if arcpy.Exists(obstruction_fc):
        desc = arcpy.Describe(obstruction_fc)
        obstruction_spatial_ref = desc.spatialReference

        with arcpy.da.SearchCursor(obstruction_fc, ["SHAPE@"]) as cursor:
            for obstruction in cursor:
                obstruction_geom = obstruction[0]
                observer_geom = arcpy.PointGeometry(arcpy.Point(obs_lon, obs_lat), spatial_ref_wgs84)

                # Project both points to the obstruction spatial reference
                obstruction_projected = obstruction_geom.projectAs(obstruction_spatial_ref)
                observer_projected = observer_geom.projectAs(obstruction_spatial_ref)

                # Compute distance in feet
                first_obstruction_distance_ft = meters_to_feet(observer_projected.distanceTo(obstruction_projected))
                break

        # Update result with obstruction distance
        visibility_results[-1] = (name, tar_is_visible, shape_length_ft, first_obstruction_distance_ft)

# Print LOS analysis results
if visibility_results:
    print("\n🔹 Line of Sight Analysis Results (in feet):")
    for name, tar_is_visible, shape_length_ft, obstruction_distance_ft in visibility_results:
        status = "Visible" if tar_is_visible == 1 else "Obstructed"
        obs_text = f", First Obstruction at {obstruction_distance_ft:.2f} ft" if obstruction_distance_ft else ""
        print(f"{name}: {status}, LOS Length = {shape_length_ft:.2f} ft{obs_text}")
else:
    print("⚠ No sight lines were generated.")

print("\n✅ LOS Analysis Complete. Results stored in your ArcGIS workspace.")



🔹 Line of Sight Analysis Results (in feet):
1_SDR: Obstructed, LOS Length = 281.35 ft
1_SDR: Obstructed, LOS Length = 772.30 ft, First Obstruction at 135.94 ft
1_SDL: Obstructed, LOS Length = 119.22 ft
1_SDL: Obstructed, LOS Length = 822.49 ft, First Obstruction at 76.25 ft
2_SDR: Obstructed, LOS Length = 105.00 ft
2_SDR: Obstructed, LOS Length = 833.89 ft, First Obstruction at 91.42 ft
2_SDL: Obstructed, LOS Length = 71.13 ft
2_SDL: Obstructed, LOS Length = 869.14 ft, First Obstruction at 31.23 ft
3_SDR: Obstructed, LOS Length = 260.78 ft
3_SDR: Obstructed, LOS Length = 507.51 ft, First Obstruction at 297.28 ft
3_SDL: Obstructed, LOS Length = 634.01 ft
3_SDL: Obstructed, LOS Length = 318.30 ft, First Obstruction at 937.74 ft
4_SDR: Obstructed, LOS Length = 156.89 ft
4_SDR: Obstructed, LOS Length = 777.92 ft, First Obstruction at 122.09 ft
4_SDL: Obstructed, LOS Length = 156.19 ft
4_SDL: Obstructed, LOS Length = 729.37 ft, First Obstruction at 117.29 ft
5_SDR: Obstructed, LOS Length =